# Convolutional Neural Networks
---

Nessa aula vamos treinar uma rede CNN para classificar imagens do dataset CIFAR-10 utilizando aumento de dados

O dataset e composto de imagens coloridas divididas em 10 classes, como mostrado no exemplo:

<img src='assets/cifar_data.png' width=70% height=70% />

### Testando o [CUDA](http://pytorch.org/docs/stable/cuda.html)

Como essas imagens sao maiores (32x32x3), pode ser muito util acelerar o processo de treinamento usando processamento em placa de videos. CUDA e uma plataforma de computacao paralela que utiliza tensores (algo tipo os arrays do numpy) para computar informacoes na GPU.

In [1]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA nao funcionando. Treinamento na CPU ...')
else:
    print('CUDA esta disponivel! Treinando na GPU ...')

---
## Carregando e aumentando os [dados](https://pytorch.org/vision/stable/datasets.html)

Carregar o dataset pode levar alguns minutos. Vamos carregar os dados de treinamento e teste, dividir o conjunto de treinamento em treinamento e validacao, e criar os Dataloaders para cada um desses conjuntos.


#### Aumento de dados

Vamos tambem executar uma forma simples de [aumento de dados](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced) que inverte e rotaciona as imagens de forma aleatoria. Isso e feito definindo uma transformacao do `torchvision`. Voce pode procurar mais tipos de transformacao para pre-processamento e aumento de dados [aqui](https://pytorch.org/vision/stable/transforms.html).


Este tipo de aumento de dados deve adicionar alguma variedade em relacao a posicao das imagens, para que quando treinarmos o modelo e seja robusto em relacao a mudancas geometricas (exemplo: reconhecer um navio, independente da direcao em que ele esteja apontando). E recomendavel que voce sempre escolha uma ou duas transformacoes.

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# numero de subprocessos usados para carregar os dados
num_workers = 0
# quantas amostras serao carregadas para cada batch
batch_size = 20
# porcentagem dos conjuntotes de treinamento e validacao
valid_size = 0.2

# converte os dados para um torch.FloatTensor normalizado
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # inverte e rotaciona aleatoriamente
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# escolhe os dados de treino e teste
train_data = datasets.CIFAR10('~/.pytorch/CIFAR10_data/', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('~/.pytorch/CIFAR10_data/', train=False,
                             download=True, transform=transform)

# obtem os indices de treino que serao usados para validacao
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define os samplers para extrair os batches de treinamento e validacao
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepara os data loaders (combina o dataset e o sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# especifica a classe das imagens
classes = ['aviao', 'automovel', 'passaro', 'gato', 'veado',
           'cachorro', 'sapo', 'cavalo', 'navio', 'caminhao']

### Visualizando o batch de treinamento

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

# funcao auxiliar para desnormalizar e mostrar a imagem
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # converte o tensor para imagem

In [3]:
# obtem um batch de amostras de treinamento
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # converte as imagens para formato numpy para apresentar

# plota as imagens no batch, junto com os respectivos rotulos
fig = plt.figure(figsize=(25, 4))
# mostra 20 imagens
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

### Visualizando as imagens com maiores detalhes

Aqui nos olhamos para os canais vermelho, verde e azul (RGB) normalizados como tres canais separados de intensidade em tons de cinza.

In [4]:
rgb_img = np.squeeze(images[3])
channels = ['canal vermelho', 'canal verde', 'canal azul']

fig = plt.figure(figsize = (36, 36)) 
for idx in np.arange(rgb_img.shape[0]):
    ax = fig.add_subplot(1, 3, idx + 1)
    img = rgb_img[idx]
    ax.imshow(img, cmap='gray')
    ax.set_title(channels[idx])
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            val = round(img[x][y],2) if img[x][y] !=0 else 0
            ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center', size=8,
                    color='white' if img[x][y]<thresh else 'black')

---
## Definindo a  [arquitetura](http://pytorch.org/docs/stable/nn.html) da rede

Agora vamos definir a arquitetura da CNN. Em vez de camadas lineares, como nas redes MLP, vamos usar as seguintes camadas:
* [Camadas convolucionais](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d), as quais podem ser consideradas um empilhamento de imagens filtradas.
* [Camadas Maxpooling](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d), as quais reduzem as dimensoes de uma entrada, mantendo apenas os pixels mais ativos da camada anterior.
* As ja conhecidas camadas lineares e Dropout para evitar overfitting e produzir uma saida de dimensionalidade igual a 10 (numero de classes).

A imagem abaixo apresenta 2 camadas convolucionais.

<img src='assets/2_layer_conv.png' height=50% width=50% />

#### Definindo um modelo com multiplas camadas convolucionais e definindo o comportamento do modelo durante a passagem feedforward.

Note que quanto mais camadas convolucionais forem incluidas, mais complexos sao os padroes de cor e forma que o modelo e capaz de detectar. Para essa tarefa, e sugerido um modelo com 2 ou 3 camadas convolucionais, com camadas lineares e dropout para evitar overfitting. Para termos uma idea de como escolher esses valores, uma boa sugestao seria procurar trabalhos ja publicados. 


In [8]:
import torch.nn as nn
import torch.nn.functional as F

# Definindo nossa arquitetura
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Camada convolucional (enxerga um tensor representando uma imagem de dimensao 32x32x3)
        # primeiro parametro e o numero de canais de entrada (3 pq e RGB), segundo e o numero de canais de saida (16)
        # terceiro e o tamanho do kernel (filtro de 3x3) e o padding e uma moldura para evitar que a imagem seja redimensionada.
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # Camada convolucional (enxerga um tensor de  16x16x16 - 16 canais, dimensionalidade reduzida de 32x32 pela camada de MaxPool)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # Camada convolucional (enxerga um tensor de  8x8x32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # camada max pooling
        self.pool = nn.MaxPool2d(2, 2)
        # camada linear (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # camada linear (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # camada dropout (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # adiciona uma sequencia de convolucoes e maxpooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # achata x, transformando em um vetor
        x = x.view(-1, 64 * 4 * 4)
        # adiciona a camada dropout
        x = self.dropout(x)
        # adiciona a primeira camada escondida, com ativacao relu
        x = F.relu(self.fc1(x))
        # adiciona uma camada dropout
        x = self.dropout(x)
        # adiciona segunda camada escondida, com ativacao relu
        x = self.fc2(x)
        return x

# cria uma CNN completa
model = Net()
print(model)

# move os tensores pra GPU, se o CUDA estiver disponivel
if train_on_gpu:
    model.cuda()

### Definindo a  [funcao de custo](https://pytorch.org/docs/stable/nn.html#loss-functions)

Precisamos decidir qual a melhor funcao de custo e otimizacao para a tarefa de classificacao.Precisamos tamber prestar atencao ao valor da **taxa de aprendizado**, uma vez que ela determina como o modelo vai convergir para um erro menor.

In [11]:
import torch.optim as optim

# especificamos como funcao de custo a funncao categorical cross-entropy
criterion = nn.CrossEntropyLoss()

# e como funcao de otimizacao o gradiente descendente estocastico
optimizer = optim.SGD(model.parameters(), lr=0.01)

---
## Treinando a rede

Lembrem que precisamos verificar como o custo de treinamento e validacao diminuem conforme o tempo. Se o custo de validacao aumenta, pode indicar overfitting.

In [11]:
# numero de epocas de treinamento
n_epochs = 12

valid_loss_min = np.Inf # mantem registro do custo minimo de validacao

for epoch in range(1, n_epochs+1):

    # mantem registro dos custos de treinamento e validacao
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # treina o modelo #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move os tensores pra GPU se o CUDA estiver disponivel
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # limpa os gradientes de todas as variaveis otimizadas
        optimizer.zero_grad()
        # passo forward: computa as saidas do modelo
        output = model(data)
        # calcula o custo por batch
        loss = criterion(output, target)
        # passo backward: computa o gradiente do custo em respeito aos parametros do modelo
        loss.backward()
        # executa um unico passo de otimizacao (atualizacao dos parametros)
        optimizer.step()
        # atualiza o custo de trein
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # valida o modelo #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move os tensores para a GPU se o CUDA estiver disponivel
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # passo forward: computa as saidas do modelo
        output = model(data)
        # calcula o custo por batch
        loss = criterion(output, target)
        # atualiza o custo de validacao
        valid_loss += loss.item()*data.size(0)
    
    # calcula a media dos custos
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # imprime  as estatisticas de treino e validacao 
    print('Epoca: {} \tCusto de treinamento: {:.6f} \tCusto de validacao: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # salva o modelo se o custo de validacao diminuir
    if valid_loss <= valid_loss_min:
        print('Reducao do custo de validacao ({:.6f} --> {:.6f}).  Salvando o modelo ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

### Carregando o modelo com menor custo de validacao

In [12]:
model.load_state_dict(torch.load('model_augmented.pt'))

---
## Testando a rede treinada

Testa seu modelo treinado em dados novos. Um "bom" resultado sera uma CNN com acuracia por volta de 70% (ou mais, tente melhorar) sobre as imagens de teste.

In [13]:
# registra o custo de teste
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# itera sobre os dados de teste
for batch_idx, (data, target) in enumerate(test_loader):
    # move os tensores pra GPU, se disponivel
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # passo forward: computa as saidas do modelo
    output = model(data)
    # computa o custo do batch
    loss = criterion(output, target)
    # atualiza o custo de teste
    test_loss += loss.item()*data.size(0)
    # converte as probabilidades para a classe predita
    _, pred = torch.max(output, 1)    
    # compara as predicoes com os rotulos reais
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # computa a acuracia para cada classe
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# media de custo de teste
test_loss = test_loss/len(test_loader.dataset)
print('Custo de teste: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Acuracia de teste de %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Acuracia de teste de %5s: N/A (nenhuma amostra vista no treinamento)' % (classes[i]))

print('\nAcuracia de teste (Geral): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### Visualizando os resultados de teste

In [14]:
# obtendo um batch de amostras de teste
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# movendo as entradas para GPU
if train_on_gpu:
    images = images.cuda()

# obtendo as saidas
output = model(images)
# convertendo as probabilidades em classes
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plotando as imagens do batch junto com a classe predita e real
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx].to('cpu'))
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))

# Exercicios

1. Pesquise a [documentacao sobre transformacoes](https://pytorch.org/vision/stable/transforms.html) e adicione outros tipos de transformacao para aumento de dados e veja como o modelo vai se comportar.

2. Tentar modificar o modelo para conseguir uma acuracia maior sobre o conjunto de teste.